## Doramas Báez Bernal - Implementacion de métodos de imputación

La tarea a desarrollar consiste en la implementación y comparación de dos métodos de imputación de valores perdidos estudiados en clase. En concreto,vamos a tratar de analizar el método MICE utilizando como estimador la regresión lineal y el método Hot-Deck por muestra más próxima.

Para ello, se utilizará el conjunto de datos winequality‐white‐missing.csv que contiene valores perdidos y se comparará los valores obtenidos de las distintas imputaciones utilizando el conjunto winequality‐white.csv. Además, se utilizará el error cuadrático medio como medida de comparacion de los resultados.

### Objetivos
- Descripción de los datos.
- Implementar método MICE.
- Implementar método Hot-Deck.
- Aplicar los métodos desarrollados en clase.
- Comparar los resultados.

In [1]:
import pandas as pd
import numpy as np 
import random 
from sklearn import linear_model, metrics 
import pandas_profiling

In [2]:
datos = pd.read_csv('data/winequality-white-missing.csv', sep=';', dtype={'quality':object})
datos_original = pd.read_csv('data/winequality-white.csv', sep=';', dtype={'quality':object})
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4400 non-null   float64
 1   volatile acidity      4431 non-null   float64
 2   citric acid           4411 non-null   float64
 3   residual sugar        4434 non-null   float64
 4   chlorides             4387 non-null   float64
 5   free sulfur dioxide   4398 non-null   float64
 6   total sulfur dioxide  4429 non-null   float64
 7   density               4413 non-null   float64
 8   pH                    4415 non-null   float64
 9   sulphates             4372 non-null   float64
 10  alcohol               4412 non-null   float64
 11  quality               4898 non-null   object 
dtypes: float64(11), object(1)
memory usage: 459.3+ KB


Se dispone de un conjunto de datos que indica la calidad de unos vinos. Para ello, disponemos de ciertas caracteristicas que pueden definir a los vinos como son el azucar residual, el acido citrico o la cantidad de alcohol. En general, el conjunto de datos consta de 10 características de valores reales y una caracteristica utilizada para indicar la calidad del vino, esta no es continua, pero puede ser tratada como valores real a la hora de realizar los cálculos. 

Otro aspecto que se puede observar es que el dataset consta de 4898 muestras. Sin embargo, se puede observar la existencia de columnas que contienen valores menores, como puede ser la columna "fixed acidity". Esto significa, que hay existencia de valores perdidos para esa columna, por lo tanto, se deberá decidir que hacer con estos valores perdidos. En este caso, se va a aplicar diversas tecnicas de imputacion para intentar obtener un conjunto de datos lo mayor posible.



## MICE ALGORITHM

1\. Se realiza una imputación simple en cada variable con valores perdidos.

2\. Se seleciona una variable x con valor perdido y se retorna a valor perdido.

3\. Se realiza una estimacion mediante regresion lineal, utilizando el resto de variables.

4\. El valor perdido de x, se sustituye por la estimacion obtenida en el paso 3.

5\. Se repite los pasos para todas las variables con valores perdidos (un ciclo).

6\. Repetir un determinado número de ciclos.



In [3]:
def Mice(dataset,iterations):
    datos_Mice = dataset.copy()
    colums_names = list(datos_Mice.columns)

    # Se calcula la media para la imputacion de valores perdidos
    for i in range(11):
            variables_perdidas = dataset.iloc[:,i].isnull() # Se obtienen valores nulos del conjunto de datos original
            media_variables = datos_Mice.loc[~variables_perdidas,colums_names[i]].mean()
            datos_Mice.loc[variables_perdidas,colums_names[i]] = media_variables

    for _ in range(iterations):
        # Prediccion por regresion lineal
        for j in range(11):
            variables_perdidas = dataset.iloc[:,j].isnull() 
            datos_Mice.loc[variables_perdidas,colums_names[j]] = np.nan # Paso 2 MICE

            matrix_perdidos = datos_Mice.loc[variables_perdidas].values
            matrix_no_perdidos = datos_Mice.loc[~variables_perdidas].values

            x_perdidos = np.delete(matrix_perdidos,j,axis=1)
            x = np.delete(matrix_no_perdidos,j,axis=1)
            y = matrix_no_perdidos[:,j]

            modelo_regresion = linear_model.LinearRegression()
            modelo_regresion.fit(x,y)   

            variables_pred = modelo_regresion.predict(x_perdidos)  
            datos_Mice.loc[variables_perdidas,colums_names[j]] = variables_pred
    
    
        
    return datos_Mice


In [4]:
res_mice = Mice(datos,20)
colums_names = list(res_mice.columns)

for i in range(11):
    variables_perdidas_originales = datos.iloc[:,i].isnull()
    mse = metrics.mean_squared_error(res_mice.loc[variables_perdidas_originales,colums_names[i]],                datos_original.loc[variables_perdidas_originales,colums_names[i]] )

    print("El error medio en la columna " + str(colums_names[i]) + " es " + str(mse))

        

El error medio en la columna fixed acidity es 0.3757166302748024
El error medio en la columna volatile acidity es 0.009578597026662396
El error medio en la columna citric acid es 0.010867231189791634
El error medio en la columna residual sugar es 3.2950084324021143
El error medio en la columna chlorides es 0.0002190964280252457
El error medio en la columna free sulfur dioxide es 229.56159758464275
El error medio en la columna total sulfur dioxide es 953.962289072321
El error medio en la columna density es 6.80636695602238e-07
El error medio en la columna pH es 0.011872297720247196
El error medio en la columna sulphates es 0.011400034090210214
El error medio en la columna alcohol es 0.2983388316017732


## Hot-Deck algorithm

Consiste en sustituir los valores perdidos por el valor que posee la muestra más proxima considerando todos los atributos excepto el que contiene valores perdidos.
- Genera siempre el mismo valor
- No es necesario estimar parámetros


In [5]:
import scipy


def Hot_Deck(dataset):
    datos_Hot_Deck = dataset.copy()
    colums_names = list(dataset.columns)

    rows_with_nan = datos_Hot_Deck[datos_Hot_Deck.isnull().any(axis=1)]
    
    rows_without_nan = datos_Hot_Deck.dropna()

    for i in range(len(rows_with_nan)): 

        row_null = rows_with_nan.iloc[i,:].isnull()
        row_aux = rows_with_nan.iloc[i,:]

        row_clean = row_aux.loc[~row_null].values.reshape(1,-1)
        
        matrix_without_nan = rows_without_nan.loc[:,~row_null].values
        

        x = scipy.spatial.distance.cdist(row_clean, matrix_without_nan, 'euclidean')
        result = np.where(x == np.amin(x))

        for x_loop in range(11):
            if row_null[x_loop] == True:
                rows_with_nan.iloc[i,x_loop] = rows_without_nan.iloc[result[1][0],x_loop]
                
            
    res = rows_with_nan.append(rows_without_nan)
    return res  


In [6]:
res_hot_deck = Hot_Deck(datos)
colums_names = list(res_mice.columns)

for i in range(11):
    variables_perdidas_originales = datos.iloc[:,i].isnull()
    mse = metrics.mean_squared_error(res_hot_deck.loc[variables_perdidas_originales,colums_names[i]],                datos_original.loc[variables_perdidas_originales,colums_names[i]] )

    print("El error medio en la columna " + str(colums_names[i]) + " es " + str(mse))

El error medio en la columna fixed acidity es 1.2949397590361449
El error medio en la columna volatile acidity es 0.015336241970021416
El error medio en la columna citric acid es 0.022559342915811086
El error medio en la columna residual sugar es 26.992516163793105
El error medio en la columna chlorides es 0.0004246712328767123
El error medio en la columna free sulfur dioxide es 310.1065
El error medio en la columna total sulfur dioxide es 1639.9962686567164
El error medio en la columna density es 2.664641701030923e-06
El error medio en la columna pH es 0.034687370600414075
El error medio en la columna sulphates es 0.02003555133079848
El error medio en la columna alcohol es 1.4119821673525383


## Imputacion por sustitucion por la media

Uno de los métodos con los que vamos a comparar los algoritmos implementados anteriormente es la imputacion por media, así poder cuantificar la calidad de estos.


In [7]:
def imputation_mean(dataset):
    datos1 = dataset.copy()
    colums_names = list(datos1.columns)

    for i in range(11):
        variables_perdidas = datos1.iloc[:,i].isnull() 
        media_alcohol = datos1.loc[~variables_perdidas,colums_names[i]].mean() 
        datos1.loc[variables_perdidas,colums_names[i]] = media_alcohol

    return datos1


## Imputacion por regresión lineal 

Otro de los métodos con los que compararemos el algoritmo Mice y Hot_Deck, será la imputación por regresión lineal. Esta como veremos más adelante, dará unos resultados muy satisfactorios. Sin embargo, no es posible realizarlo en un entorno real, ya que, necesita no tener valores nulos excepto en la columna a imputar.

In [8]:
def linear_regresion(dataset):
    datos1 = dataset.copy()
    colums_names = list(dataset.columns)

    for x_loop in range(11):
        datos1[colums_names[x_loop]] = datos[colums_names[x_loop]]
        variables_perdidas = datos1[colums_names[x_loop]].isnull()


        matrix_perdidos = datos1.loc[variables_perdidas].values
        matrix_no_perdidos = datos1.loc[~variables_perdidas].values
        

        x_perdidos = np.delete(matrix_perdidos,x_loop,axis=1)
        x = np.delete(matrix_no_perdidos,x_loop,axis=1)
        y = matrix_no_perdidos[:,x_loop]

        modelo_regresion = linear_model.LinearRegression()
        modelo_regresion.fit(x,y)   

        variables_pred = modelo_regresion.predict(x_perdidos)  
        datos1.loc[variables_perdidas,colums_names[x_loop]] = variables_pred

    return datos1  


## Comparar los resultados

En esta sección, se compararan los resultados de los errores medios de los métodos anteriores y se intentará argumentar las diferencias que se producen entre los mismos.

In [9]:
res_mice = Mice(datos,20)
    
res_hot_deck = Hot_Deck(datos)
res_mean = imputation_mean(datos) 
res_lineal = linear_regresion(datos_original)

colums_names = list(datos.columns)

for i in range(11):
    variables_perdidas_originales = datos.iloc[:,i].isnull()

    mse_mice = metrics.mean_squared_error(res_mice.loc[variables_perdidas_originales,colums_names[i]],                datos_original.loc[variables_perdidas_originales,colums_names[i]] )

    mse_deck = metrics.mean_squared_error(res_hot_deck.loc[variables_perdidas_originales,colums_names[i]],                datos_original.loc[variables_perdidas_originales,colums_names[i]] )

    mse_mean = metrics.mean_squared_error(res_mean.loc[variables_perdidas_originales,colums_names[i]],                datos_original.loc[variables_perdidas_originales,colums_names[i]] )

    mse_lineal = metrics.mean_squared_error(res_lineal.loc[variables_perdidas_originales,colums_names[i]],                datos_original.loc[variables_perdidas_originales,colums_names[i]] )

    print("Error medio en la columna " + str(colums_names[i]) + " para el algoritmo mice es " + str(mse_mice))
    print("Error medio en la columna " + str(colums_names[i]) + " para el algoritmo hot_deck es " + str(mse_deck))
    print("Error medio en la columna " + str(colums_names[i]) + " para el algoritmo mean es " + str(mse_mean))
    print("Error medio en la columna " + str(colums_names[i]) + " para el algoritmo regresion lineal es " + str(mse_lineal))
    print("===========================================================")

Error medio en la columna fixed acidity para el algoritmo mice es 0.3757166302748024
Error medio en la columna fixed acidity para el algoritmo hot_deck es 1.2949397590361449
Error medio en la columna fixed acidity para el algoritmo mean es 0.8310362388028355
Error medio en la columna fixed acidity para el algoritmo regresion lineal es 0.30715519384607953
Error medio en la columna volatile acidity para el algoritmo mice es 0.009578597026662396
Error medio en la columna volatile acidity para el algoritmo hot_deck es 0.015336241970021416
Error medio en la columna volatile acidity para el algoritmo mean es 0.011386864703054672
Error medio en la columna volatile acidity para el algoritmo regresion lineal es 0.009409758339209361
Error medio en la columna citric acid para el algoritmo mice es 0.010867231189791634
Error medio en la columna citric acid para el algoritmo hot_deck es 0.022559342915811086
Error medio en la columna citric acid para el algoritmo mean es 0.011884959761715051
Error me

- Imputación por Mice: en general, los resultados obtenidos mediante la imputacion son bastante buenos, se puede observar que son similares a la regresion lineal y esto es significativo porque indica que se obtiene buena calidad en los resultados. Además, un aspecto muy importante es que para realizar imputacion mediante el método mice no dependemos de los datos originales.

- Imputación por Hot-Deck: se puede observar, que para el algoritmo Hot_Deck, a veces se obtiene unos valores buenos y otras veces no tanto. Esto es debido a que, se ha divido el dataset en dos conjuntos, filas sin valores nulos y filas con valores nulos. Por lo tanto, cuando se realiza el calculo euclideo para calcular las distancias, estamos comparando la fila que tiene un valor nulo con las filas sin valores nulos. Esto provoca, que se pierda una gran cantidad de posibles vecinos y que la distancia minima entre los vecinos pueda ser elevada en algunos casos. No obstante, se puede observar que cuando encuentra vecinos buenos este algoritmo es bastante aceptable aunque un poco peor que el mice. 

- Imputacion por media: la imputacion por media simple, suele dar unos errores bastante elevados y normalmente se debe descartar utilizar dicho metodo.

- Imputación por regresion lineal: este método da los mejores resultados, sin embargo, este es solo aplicable a un caso ideal donde se dispone el conjunto de datos original.

